# [TFRecordとtf.train.Example](https://www.tensorflow.org/tutorials/load_data/tfrecord)

TFRecordはバイナリレコードの配列を保存する簡単なフォーマットである。[Protocol buffers](https://developers.google.com/protocol-buffers/)はプラットフォーム横断、言語横断可能なライブラリで、構造データを効率的にシリアル化できる。プロトコルのメッセージは`.proto`ファイルに定義されており、メッセージのタイプを知るのに最も簡単な方法である。

`tf.train.Example`メッセージ（またはprotobuf）は柔軟なメッセージタイプで`{"string": value}`マッピングを表す。TensorFlowやハイレベルなAPIで利用することを想定してデザインされている。

ここでは、`tf.train.Example`メッセージをどのように作成、分析、利用するか、そしてそのメッセージをどのようにシリアル化、書き込み、読み込みを行うかを実験する。

In [1]:
import tensorflow as tf

import numpy as np
import IPython.display as display

## `tf.train.Example`

基本的に`tf.train.Example`は`{"string": tf.train.Feature}`のマップである。`tf.train.Feature`のメッセージタイプは以下の三つのタイプのうち一つを受け取れる。汎用なデータタイプはこれらのデータ型に変換できる。

1. `tf.train.ByteList`: `string`, `byte`はこの型に変換
2. `tf.train.FloatList`: `float32`, `float64`はこの型に変換
3. `tf.train.Int64List`: `bool`, `enum`, `int32`, `uint32`, `int64`, `uint64`はこの型に変換

TensorFlowの標準なデータ型を`tf.train.Example`互換の`tf.train.Feature`に変換するためには、以下のショートカット関数を利用することができる。それぞれの関数はスカラー値を入力として受け取り、上の型のどれかを含む`tf.train.Feature`を返す。

In [2]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
print(_bytes_feature(b'test_string'))
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

bytes_list {
  value: "test_string"
}

bytes_list {
  value: "test_bytes"
}



In [4]:
print(_float_feature(np.exp(1)))

float_list {
  value: 2.7182817459106445
}



In [5]:
print(_int64_feature(True))
print(_int64_feature(1))

int64_list {
  value: 1
}

int64_list {
  value: 1
}



In [6]:
# All proto messages can be serialized to a binary-string.
feature = _float_feature(np.exp(1))
feature.SerializeToString()

b'\x12\x06\n\x04T\xf8-@'

`tf.train.Example`メッセージを既存のデータから作成することを考える。一つの観測データからこれを作成するには以下の手順に従う。

1. 各観測データ内にある各値を`tf.train.Feature`に変換する（定義した関数を使う）
2. 特徴名からStep 1でエンコードした特徴量への辞書マップを作成する
3. Step 2で作成した辞書マップを`Feature`メッセージに変換する

ここではNumPyを使ってデータセットを作成する。このデータセットは以下の4つの特徴量を持つ。

- 同数ずつの`False`と`True`のboolean型
- `[0, 5]`からランダムに選んだ整数型
- 整数型の特徴量をインデックスとして使って、文字列テーブルから作成した文字列型
- 標準正規分布から得たフロート型

In [7]:
# The number of observations in the dataset.
n_observations = int(1e4)

# Boolean feature, encoded as False or True.
feature0 = np.random.choice([False, True], n_observations)

# Integer feature, random from 0 to 4.
feature1 = np.random.randint(0, 5, n_observations)

# String feature
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]

# Float feature, from a standard normal distribution
feature3 = np.random.randn(n_observations)

これらの特徴量はそれぞれ`tf.train.Example`互換型に変換することができる。これには`_bytes_feature`, `_float_feature`, `_int64_feature`を使う。これらのエンコードした特徴量から`tf.train.Example`メッセージを作成する。

In [8]:
def serialize_example(feature0, feature1, feature2, feature3):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.
    feature = {
        'feature0': _int64_feature(feature0),
        'feature1': _int64_feature(feature1),
        'feature2': _bytes_feature(feature2),
        'feature3': _float_feature(feature3),
    }

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [9]:
# This is an example observation from the dataset.

example_observation = []

serialized_example = serialize_example(False, 4, b'goat', 0.9876)
serialized_example

b'\nR\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04[\xd3|?\n\x14\n\x08feature2\x12\x08\n\x06\n\x04goat\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x00\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x04'

In [10]:
# To decode the message use the `tf.train.Example.FromString` method.

example_proto = tf.train.Example.FromString(serialized_example)
example_proto

features {
  feature {
    key: "feature0"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "feature1"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "feature2"
    value {
      bytes_list {
        value: "goat"
      }
    }
  }
  feature {
    key: "feature3"
    value {
      float_list {
        value: 0.9876000285148621
      }
    }
  }
}

## TFRecordsフォーマットの詳細

TFRecordファイルはレコードのシーケンスを含み、先頭から順番に読み込まれる。それぞれのレコードは以下のフォーマットとなっている。

```record_format
uint64 length
uint32 masked_crc32_of_length
byte   data[length]
uint32 masked_crc32_of_data
```

ファイルを作成するためにはレコードを結合する。CRC：
`` masked_crc = ((crc >> 15) | (crc << 17)) + 0xa282ead8ul ``

## `tf.data`を使ったTFRecordファイル

`tf.data`モジュールはTensorFlowでデータを読み書きを行うためのツールも提供している。

TFRecordファイルの書き込み

In [11]:
# Get the data into a dataset using the `from_tensor_slices` method
features_dataset = tf.data.Dataset.from_tensor_slices((feature0, feature1, feature2, feature3))
features_dataset

<TensorSliceDataset shapes: ((), (), (), ()), types: (tf.bool, tf.int64, tf.string, tf.float64)>

In [12]:
# Use `take(1)` to only pull one example from the dataset.
for f0,f1,f2,f3 in features_dataset.take(1):
    print(f0)
    print(f1)
    print(f2)
    print(f3)

tf.Tensor(True, shape=(), dtype=bool)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(b'chicken', shape=(), dtype=string)
tf.Tensor(-0.1817618358434813, shape=(), dtype=float64)


`tf.data.Dataset.map`メソッドを使って`Dataset`の各要素に適用する。マップされる関数は`tf.Tensors`に作用し`tf.Tensors`を返す必要がある。`serialize_example`のような非テンソル関数は`tf.py_function`にラップすることができる。`tf.py_function`にはテンソルの形と型を指定する必要がある。

In [13]:
def tf_serialize_example(f0,f1,f2,f3):
    tf_string = tf.py_function(
        serialize_example,
        (f0,f1,f2,f3),  # pass these args to the above function.
        tf.string)      # the return type is `tf.string`.
    return tf.reshape(tf_string, ()) # The result is a scalar

In [14]:
tf_serialize_example(f0,f1,f2,f3)

<tf.Tensor: shape=(), dtype=string, numpy=b'\nU\n\x17\n\x08feature2\x12\x0b\n\t\n\x07chicken\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x01\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04\xc6\x1f:\xbe\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x02'>

In [15]:
# Apply this function to each element in the dataset.
serialized_features_dataset = features_dataset.map(tf_serialize_example)
serialized_features_dataset

<MapDataset shapes: (), types: tf.string>

In [16]:
def generator():
    for features in features_dataset:
        yield serialize_example(*features)

In [17]:
serialized_features_dataset = tf.data.Dataset.from_generator(
    generator,
    output_types=tf.string,
    output_shapes=()
)

In [18]:
serialized_features_dataset

<FlatMapDataset shapes: (), types: tf.string>

In [19]:
# Write them to a TFRecord file.
filename = 'test.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_features_dataset)

TFRecordファイルの読み込み（[詳細](https://www.tensorflow.org/guide/datasets#consuming_tfrecord_data)）

In [20]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

この時点でのデータセットはシリアル化した`tf.train.Example`メッセージを含む。これをイテレートした場合、文字列テンソルのスカラーを返す。

In [21]:
for raw_record in raw_dataset.take(10):
    print(repr(raw_record))

<tf.Tensor: shape=(), dtype=string, numpy=b'\nU\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04\xc6\x1f:\xbe\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x01\n\x17\n\x08feature2\x12\x0b\n\t\n\x07chicken\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x02'>
<tf.Tensor: shape=(), dtype=string, numpy=b'\nQ\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x00\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x01\n\x13\n\x08feature2\x12\x07\n\x05\n\x03dog\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04L*P\xbf'>
<tf.Tensor: shape=(), dtype=string, numpy=b'\nQ\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x00\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04\xe6\x9b\x1d\xbf\n\x13\n\x08feature2\x12\x07\n\x05\n\x03cat\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x01'>
<tf.Tensor: shape=(), dtype=string, numpy=b'\nR\n\x11\n\x08feature1\x12\x05\x1a\x03\n\x01\x04\n\x14\n\x08feature3\x12\x08\x12\x06\n\x04I]\xe3\xbe\n\x11\n\x08feature0\x12\x05\x1a\x03\n\x01\x01\n\x14\n\x08feature2\x12\x08\n\x06\n\x04goat'>
<tf.Tensor: shape=(), dtype=string, numpy=b

これらのテンソルは以下の関数を利用して分析することができる。データセットはグラフ実行を使っているため、`feature_description`が必要であり、テンソルの形と型を構築する説明が必要である。

In [22]:
# Create a description of the features.
feature_description = {
    'feature0': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature1': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature2': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'feature3': tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
}

def _parse_function(example_proto):
    # Parse the input `tf.train.Example` proto using the dictionary above.
    return tf.io.parse_single_example(example_proto, feature_description)

In [23]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

<MapDataset shapes: {feature0: (), feature1: (), feature2: (), feature3: ()}, types: {feature0: tf.int64, feature1: tf.int64, feature2: tf.string, feature3: tf.float32}>

In [24]:
for parsed_record in parsed_dataset.take(10):
    print(repr(parsed_record))

{'feature0': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'feature1': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'feature2': <tf.Tensor: shape=(), dtype=string, numpy=b'chicken'>, 'feature3': <tf.Tensor: shape=(), dtype=float32, numpy=-0.18176183>}
{'feature0': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'feature1': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'feature2': <tf.Tensor: shape=(), dtype=string, numpy=b'dog'>, 'feature3': <tf.Tensor: shape=(), dtype=float32, numpy=-0.8131454>}
{'feature0': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'feature1': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'feature2': <tf.Tensor: shape=(), dtype=string, numpy=b'cat'>, 'feature3': <tf.Tensor: shape=(), dtype=float32, numpy=-0.6156601>}
{'feature0': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'feature1': <tf.Tensor: shape=(), dtype=int64, numpy=4>, 'feature2': <tf.Tensor: shape=(), dtype=string, numpy=b'goat'>, 'feature3': <tf.Tensor: shape=(), dtype=float32, numpy=-0.44407108>}
{'feature